In [53]:
from IPython.display import clear_output
import numpy as np
import sympy as sp
from fractions import Fraction

In [54]:
from qiskit import IBMQ
from qiskit_optimization import QuadraticProgram
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import QAOA

In [55]:
#IBMQ.load_account()
#provider = IBMQ.get_provider(hub='ibm-q')

In [56]:
clear_output()

## Define Functions

In [57]:
def construct_function(kl, kh, n, h):
    function = 0
    sp.init_printing(use_unicode=True)
    
    # construct variables for each permeability
    for i in range(1,n):
        globals()['q%s' % i] = sp.symbols('q'+str(i))
        globals()['h%s' % i] = sp.symbols('q'+str(i))
        
    # for each permeability, add its contribution to the total cost function
    for i in range(2,n):
        term = ((kl+globals()['q%s' % (i-1)]*(kh-kl))*(h[i-2]-h[i-1])+(kl+globals()['q%s' % (i)]*(kh-kl))*(h[i]-h[i-1]))**2
        function = function + term
        
    # expand and simplify the cost function
    function = sp.expand(function)
    
    # convert all square terms into linear terms as qi^2 = qi = 0 or 1 for all i and simplify the cost function
    for i in range(1,n):
        function = function.subs(sp.symbols('q'+str(i))**2, (sp.symbols('q'+str(i))))
        coeffs = sp.Poly(function).as_dict()
        
    # function in sympy form and coefficients of all terms in an easily readble dictionary form returned
    return function, coeffs

In [58]:
def generate_QP(coeffs, n):
    qp = QuadraticProgram()
    for i in range(1,n):
        qp.binary_var('q'+str(i))
    constant = 0
    linear = {}
    quadratic = {}
    for key,value in coeffs.items():
        if sum(key) == 0:
            constant = float(value)
        elif sum(key) == 1:
            term = 'q'+str(np.argmax(key)+1)
            linear[term] = float(value)
        else:
            indices = [i[0] for i in np.argwhere(np.array(key)>0)]
            term = ('q'+str(indices[0]+1),'q'+str(indices[1]+1))
            quadratic[term] = float(value)
    qp.minimize(linear=linear, quadratic=quadratic, constant=constant)
    print(qp.export_as_lp_string())
    return qp

In [59]:
def hydrologic_inverse_analysis(kl,kh,n,h):
    function, coeffs = construct_function(kl,kh,n,h)
    print('Function in Sympy:', function)
    qp = generate_QP(coeffs,n)
    qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=123)
    meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
    result = meo.solve(qp)
    print('\nrun time:', result.min_eigen_solver_result.optimizer_time)
    print(result)
    k = [int(i) for i in list(kl+result.x*(kh-kl))]
    print('k_res: ', k)
    return k

In [60]:
def evaluate_performance(k_true,k_res):
    total_number = len(k_true)
    false_count = 0
    for i in range(len(k_true)):
        if k_true[i] != k_res[i]:
            false_count += 1
    error_rate = false_count / total_number
    print('Error Rate = '+ str(error_rate*100)+'%')
    return error_rate

## Small 1D Problem

Given: 3 heads with $h1=1, h2=\frac{1}{3}, h3=0$ <br>
Assumption: $k_l = 1, k_h = 2$ <br>
Task: minimize  $f(q1, q2) = 8q_1/9 − q_2/9 − 4q_1q_2/9 + 1/9$

In [61]:
k_true = [1,2]

In [62]:
kl = 1
kh = 2
n = 3
h = [1,1/3,0]

In [63]:
k_res = hydrologic_inverse_analysis(kl,kh,n,h)

Function in Sympy: -0.444444444444444*q1*q2 + 0.888888888888889*q1 - 0.111111111111111*q2 + 0.111111111111111
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 0.888888888889 q1 - 0.111111111111 q2 + [ - 0.888888888889 q1*q2 ]/2 +
      0.111111111111
Subject To

Bounds
 0 <= q1 <= 1
 0 <= q2 <= 1

Binaries
 q1 q2
End


run time: 0.019388198852539062
optimal function value: 2.7755575615628914e-17
optimal value: [0. 1.]
status: SUCCESS
k_res:  [1, 2]


In [64]:
error_rate = evaluate_performance(k_true, k_res)

Error Rate = 0.0%


## Larger 1D Problem

In [78]:
def generate_random_1D_data(kl, kh, n):
    k_true = np.random.randint(2, size=n-1)*(kh-kl) +kl
    
    h1=np.random.rand()
    h2=np.random.rand()
    h=np.array([h1,h2])
    for i in range(1,n-1):
        h_next = h[i]+k_true[i-1]/k_true[i]*(h[i]-h[i-1])
        h=np.append(h, np.array([h_next]))
    return k_true, h

In [79]:
kl = 1
kh = 2
n = 15

In [82]:
k_true, h = generate_random_1D_data(kl, kh, n)
print("k =", k_true)
print("h =", h)

k = [1 1 1 1 2 2 2 2 1 1 2 2 2 1]
h = [0.21269449 0.64007633 1.06745816 1.49484    1.92222184 2.13591276
 2.34960368 2.56329459 2.77698551 3.20436735 3.63174919 3.84544011
 4.05913102 4.27282194 4.70020378]


In [83]:
k_res = hydrologic_inverse_analysis(kl,kh,n,h)

Function in Sympy: -0.36531046949032*q1*q2 + 0.18265523474516*q1 - 0.18265523474516*q10*q11 - 0.36531046949032*q10*q9 + 0.547965704235479*q10 - 0.0913276173725797*q11*q12 + 1.94289029309402e-16*q11 - 0.0913276173725795*q12*q13 + 0.0913276173725793*q12 - 0.182655234745159*q13*q14 + 0.365310469490318*q14 - 0.36531046949032*q2*q3 + 0.365310469490321*q2 - 0.36531046949032*q3*q4 + 0.36531046949032*q3 - 0.18265523474516*q4*q5 + 0.547965704235481*q4 - 0.0913276173725799*q5*q6 - 1.80411241501588e-16*q5 - 0.0913276173725799*q6*q7 + 0.0913276173725799*q6 - 0.0913276173725799*q7*q8 + 0.0913276173725799*q7 - 0.18265523474516*q8*q9 + 0.547965704235479*q9 + 0.18265523474516
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 0.182655234745 q1 + 0.365310469490 q2 + 0.365310469490 q3
      + 0.547965704235 q4 - 0.000000000000 q5 + 0.091327617373 q6
      + 0.091327617373 q7 + 0.547965704235 q9 + 0.547965704235 q10
      + 0.000000000000 q11 + 0.09132761

In [84]:
error_rate = evaluate_performance(k_true, k_res)

Error Rate = 0.0%


## 1D Problem Performance Analysis

In [70]:
num_heads = np.arange(2,15,14)
error_rate = []
for n in range(2,15):
    kl = 1
    kh = 2
    k_true = np.random.randint(2, size=n-1)*(kh-kl) +kl
    h1=np.random.rand()
    h2=np.random.rand()
    h=np.array([h1,h2])
    for i in range(1,n-1):
        h_next = h[i]+k_true[i-1]/k_true[i]*(h[i]-h[i-1])
        h=np.append(h, np.array([h_next]))
    k_res = hydrologic_inverse_analysis(kl,kh,n,h)
    error_rate.append(evaluate_performance(k_true, k_res))

GeneratorsNeeded: can't initialize from 'dict' without generators

Error Rate = 0.0%
